In [1]:
import os
import re
import csv
import sys
import ast 
import json
import time
import urllib
import random
import spotipy
import requests
import datetime
import numpy as np
import pandas as pd
import urllib.parse
from bs4 import BeautifulSoup
from user_agent import generate_user_agent
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# 原始所有歌手的清單
df_artists = pd.read_excel(r'csv/20200303/artist_list.xlsx')
df_artists

,Artist
0,5 Seconds of Summer
1,Adam Lambert
2,Adam Levine
3,Adele
4,Alan Walker
...,...
65,Troye Sivan
66,Vanessa Hudgens
67,Why Don't We
68,Willow Smith


In [41]:
# spotifyAPI存取權限
# access token要隨時更新 (https://spotify-token-finder.herokuapp.com/)
client_id = '1a23ed1ca359485ab81d524206dba0f6'
client_secret = 'af0e14fdb19a49dfb19ef0c1ea1d42ea'
username = 'qnwv65t11cplaz4dikhl4mjgi'
access_token = 'BQD6IPp38rExSVmqojfSeBaBlL1oqh90cKKlfb4WRnI4GZOwE4rAFC6U6C-CB7uMTRgHHwnffGlyGCUsaS4'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
# 取得歌手資訊
def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    # 歌手有items資訊
    if items:
        data = items[0]
        if len(items) > 0:
            artist_name = items[0]['name']
            artist_id = items[0]['id']
            artist_url = items[0]['external_urls']['spotify']
            artist_info = (artist_name,artist_id,artist_url)
            #print(artist_info)
            filename = 'csv/20200303/albums/df_albums_' + artist_info[0] + '.csv'
            return items[0], filename
        else:
            return None
    # 歌手沒有items資訊，則回傳check值為1
    else:
        print('None items')
        check = 1
        return check
        pass

In [11]:
# 取得歌手名稱和類型
# https://github.com/Leats/spotify-historymaker/blob/master/spotifyhistorymaker.py
# Bruno Mars = 0du5cEVh5yTK9QJze8zA0C / Adele = 4dpARuHxo51G3z768sgnrY
def get_artist_name_genres(access_token: str, artist_id: str):    
    bearer_token = f'Bearer {access_token}'
    track_headers = {'Accept': 'application/json','Content-Type': 'application/json','Authorization': bearer_token}
    response = requests.get(f'https://api.spotify.com/v1/artists/{artist_id}', headers=track_headers)
    response.raise_for_status()
    getname = response.json()['name'] # 型態str
    getid = response.json()['id'] # 型態str
    getgenres = response.json()['genres'] # 型態list
    getgenres2 = ','.join(getgenres) # 型態str
    
    list_artist_name = []
    list_artist_id = []
    list_artist_genres = []
    list_artist_name.append(getname)
    list_artist_id.append(getid)
    list_artist_genres.append(getgenres2)
    return list_artist_genres

In [42]:
# 輸出歌手名稱+Id+Genres的csv
with open("csv/20200303/artist_list_ID+Genres.csv", "w", encoding="utf_8_sig", newline='') as csvfile:
    writer = csv.writer(csvfile)
    #第一次寫入
    writer.writerow(['Artist','Id','Genres'])
    df_artists = pd.read_excel(r'csv/20200303/artist_list.xlsx')
    df = df_artists[['Artist']]
    L_artist_name = []
    L_artist_id = []
    L_artist_genres = []
    for i in range(len(df)):
        artist_name = df['Artist'][i]
        try:
            artist_info, filename = get_artist(artist_name)
            artist_id = artist_info['id']
            #print(artist_id)
            list_artist_genres = get_artist_name_genres(access_token,artist_id)
            artist_genres = ",".join(list_artist_genres) # list >>>str
            L_artist_name.append(artist_name)
            L_artist_id.append(artist_id)
            L_artist_genres.append(list_artist_genres)
        except:
            L_artist_id.append('')
            L_artist_genres.append('')
            pass
        #之後寫入
        writer.writerow([artist_name, artist_id, artist_genres])

In [86]:
# 要保留的歌手類型清單
df_genres_select = pd.read_excel(r'csv/20200303/artist_genres_select.xlsx')
df_genres_select.drop('Unnamed: 0',axis=1,inplace=True)
df_genres_select

,Genres
0,pop
1,dance pop
2,post-teen pop
3,urban contemporary
4,indie poptimism
...,...
1725,swedish choir
1726,cathedral choir
1727,atmosphere
1728,wind quintet


In [106]:
# 讀取artist_list_ID+Genres.csv
df_id_genres = pd.read_csv(r'csv/20200303/artist_list_ID+Genres.csv')
df_id_genres

,Artist,Id,Genres
0,5 Seconds of Summer,5Rl15oVamLq7FbSb0NNBNy,"boy band,dance pop,pop,post-teen pop"
1,Adam Lambert,6prmLEyn4LfHlD9NnXWlf7,"dance pop,idol,pop,post-teen pop"
2,Adam Levine,4bYPcJP5jwMhSivRcqie2n,pop rock
3,Adele,4dpARuHxo51G3z768sgnrY,"british soul,pop,uk pop"
4,Alan Walker,7vk5e3vY1uw9plTHJAMwjN,electro house
...,...,...,...
65,Troye Sivan,3WGpXCj9YhhfX11TToZcXP,"australian pop,dance pop,electropop,pop,post-t..."
66,Vanessa Hudgens,6G9bygHlCyPgNGxK2l3YdE,"dance pop,hollywood,post-teen pop,viral pop"
67,Why Don't We,2jnIB6XdLvnJUeNTy5A0J2,"boy band,pop,post-teen pop,teen pop"
68,Willow Smith,3rWZHrfrsPBxVy692yAIxF,"afrofuturism,dance pop,electropop,pop,post-tee..."


In [115]:
# 逐行查看歌手所屬類別是否在篩選後的總類別中(artist_genres_select.xlsx)
count = 0
delete_index = []
df_id_genres = pd.read_csv(r'csv/20200303/artist_list_ID+Genres.csv')
for row in range(len(df_id_genres)):
    # 把歌手類型清單的類型欄位轉換成list
    # https://www.interviewqs.com/ddi_code_snippets/get_list_pandas_datafra
    List1 = df_genres_select['Genres'].tolist() # 總類別清單
    List2 = df_id_genres['Genres'][row].split(',') # 每一位歌手的類別清單
    
    # 檢查歌手的類別有沒有包含在總類別中
    check =  any(item in List1 for item in List2)
    
    # 有包含
    if check is True:
        print(str(count) + ' : ' + 'List1 ⊄ List2' + '\n' + 'Contain Genres:' + str(list(set1.intersection(set2))) + '\n' + '-'*90)
        set1 = set(List1) 
        set2 = set(List2)
        # 原始型態set，轉成list型態，印出有交集的類別
        # https://www.geeksforgeeks.org/python-convert-set-into-a-list/
#         print('Contain Genres:',list(set1.intersection(set2)))
#         print('-'*90)   
    
    # 沒包含
    else:
        print(str(count) + ' : ' + 'List1 ⊄ List2' + '\n' + '-'*90)
        delete_index.append(count)
        pass
    count += 1

print('Delete Index:',delete_index)

0 : List1 ⊄ List2
Contain Genres:['dance pop', 'swedish pop', 'scandipop', 'post-teen pop', 'uk pop', 'pop']
------------------------------------------------------------------------------------------
1 : List1 ⊄ List2
Contain Genres:['pop', 'post-teen pop', 'boy band', 'dance pop']
------------------------------------------------------------------------------------------
2 : List1 ⊄ List2
------------------------------------------------------------------------------------------
3 : List1 ⊄ List2
Contain Genres:['dance pop', 'pop', 'post-teen pop', 'idol']
------------------------------------------------------------------------------------------
4 : List1 ⊄ List2
------------------------------------------------------------------------------------------
5 : List1 ⊄ List2
Contain Genres:['pop', 'uk pop']
------------------------------------------------------------------------------------------
6 : List1 ⊄ List2
Contain Genres:['pop']
-------------------------------------------------------

In [116]:
# 篩選後的歌手列表，輸出csv(artist_list_ID+Genres_select.csv)
# 若沒包含，則把歌手的那一列從dataframe(df_id_genres)中刪除
 # https://chrisalbon.com/python/data_wrangling/pandas_dropping_column_and_rows/
df_id_genres.drop(df_id_genres.index[[delete_count]],inplace=True)
df_id_genres.reset_index(drop=True, inplace=True)
df_id_genres.to_csv('csv/20200303/artist_list_ID+Genres_select.csv',encoding="utf_8_sig",index=0)
df_id_genres

,Artist,Id,Genres
0,5 Seconds of Summer,5Rl15oVamLq7FbSb0NNBNy,"boy band,dance pop,pop,post-teen pop"
1,Adam Lambert,6prmLEyn4LfHlD9NnXWlf7,"dance pop,idol,pop,post-teen pop"
2,Adele,4dpARuHxo51G3z768sgnrY,"british soul,pop,uk pop"
3,Alec Benjamin,5IH6FPUwQTxPSXurCrcIov,pop
4,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,"dance pop,pop,post-teen pop"
...,...,...,...
56,Troye Sivan,3WGpXCj9YhhfX11TToZcXP,"australian pop,dance pop,electropop,pop,post-t..."
57,Vanessa Hudgens,6G9bygHlCyPgNGxK2l3YdE,"dance pop,hollywood,post-teen pop,viral pop"
58,Why Don't We,2jnIB6XdLvnJUeNTy5A0J2,"boy band,pop,post-teen pop,teen pop"
59,Willow Smith,3rWZHrfrsPBxVy692yAIxF,"afrofuturism,dance pop,electropop,pop,post-tee..."


In [56]:
# 把歌手類型清單的類型欄位轉換成list
# https://www.interviewqs.com/ddi_code_snippets/get_list_pandas_dataframe
List1 = df_genres_select['Genres'].tolist()
List2 = df_test['Genres'][0].split(',')
#print('List1 =',List1)
print(len(List1))
#print('List2 =',List2)
print(len(List2))

if len(List1) == len(df_genres_select):
    print('Same')

check =  any(item in List1 for item in List2)
if check is True:
    print("List1 ⊂ List2")    
else :
    print("List1 ⊄ List2")

1730
4
Same
The list List1 contains some elements of the list List2


In [60]:
# list取交集的部分
def list_contains(List1, List2):  
    set1 = set(List1) 
    set2 = set(List2) 
    if set1.intersection(set2): 
        print('Contain Genres:',list(set1.intersection(set2))) # 原始型態set
        return True 
    else: 
        return False

List1 = df_genres_select['Genres'].tolist()
List2 = df_test['Genres'][0].split(',')
list_contains(List1, List2)

Contain Genres: ['pop', 'post-teen pop', 'boy band', 'dance pop']


True